## Build a k-table.

The k-table is the table of the maximum abundance for the minimum abundance species in a community of s species and n total abundance.

Params:
max_s: max number of species
max_n: max number of individuals

Returns:
table of ks for all combinations of s and n up to max_s and max_n.


In [144]:
import numpy as np
import math as math
import pandas as pd
import csv

In [145]:
def fill_ks(max_s, max_n):
    kmax = np.zeros(shape = [max_s, max_n + 1], dtype = int)
    
    for i in range(max_s):
        for j in range(max_n + 1):
            kmax[i, j] = math.floor(j / (i + 1))
    
    return(kmax)

In [146]:
fill_ks(5, 10)

array([[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10],
       [ 0,  0,  1,  1,  2,  2,  3,  3,  4,  4,  5],
       [ 0,  0,  0,  1,  1,  1,  2,  2,  2,  3,  3],
       [ 0,  0,  0,  0,  1,  1,  1,  1,  2,  2,  2],
       [ 0,  0,  0,  0,  0,  1,  1,  1,  1,  1,  2]])

## Build the partition counts table

Recursively build P(s, n).

Params:
max_s: max number of species
max_n: max individuals

Returns:
matrix of P(s,n) for all combinations of s and n up to max_s and max_n

Is a nightmare zone of off-by-one indexing errors, I think.

In [147]:
max_s = 5
max_n = 10

In [148]:


ps = np.full(shape = [max_s, max_n + 1], fill_value= np.nan)
ps[0, range(max_n + 1)] = 1
ps[range(max_s), 0] = 1
ps[range(max_s), 1] = 1


kmax = fill_ks(max_s, max_n)


In [149]:
# s goes from 2:max_s
# n goes from 3:max_n + 1

ss = list()
ns = list()
kmaxs = list()
ks = list()
p_lookups = list()
p_rows = list()
p_cols = list()

def p_lookup(k, s, n, ps):
    this_p = ps[s - 1, n - ((s + 1) * k) + 1]
    return(this_p)

for i in range(1, ps.shape[0]):
    
    #print(kmax[i, ])
    
    for j in range(2, ps.shape[1]):
        
        n = j - 1
        
        
        for k in range(0, kmax[i, n + 1] + 1): # here you need the +1 because range() stops at stop-1
            
            
            kmaxs.append(kmax[i,n + 1])
            ss.append(i)
            ns.append(j)
            ks.append(k)
            p_rows.append(i - 1)
            p_cols.append(n - ((i + 1) * k) + 1)
            p_lookups.append(p_lookup(k, i, n, ps))
        

ps[p_rows[92], p_cols[92]]



indices = pd.DataFrame(list(zip(ss, ns, ks, p_rows, p_cols, p_lookups)), columns = ['s', 'n', 'k', 'p_row', 'p_col', 'p_lookedup'])
indices

indices.to_csv("indices.csv", index = False)  

In [150]:
def p_over_k(s, n, kmax_table, p_table):
    sum_p = 0
    this_kmax = kmax_table[s, n + 1]
    
    for k in range(0, this_kmax + 1):
        sum_p = sum_p + p_lookup(k, s, n, p_table)
    
    return(sum_p)

In [170]:
p_over_k(0, 1, kmax, ps)

nan

In [165]:
s = 2
n = 5

sum_p = 0

this_kmax = kmax[s, n + 1]

for k in range(0, this_kmax + 1):
    sum_p = sum_p + p_lookup(k, s, n, ps)

sum_p

nan

In [173]:
sum_ps = list()

p_over_k_ss = list()
p_over_k_ns = list()

for i in range(1, ps.shape[0]):    
    for j in range(2, ps.shape[1]):
        
        n = j - 1
        
        p_over_k_ss.append(i)
        p_over_k_ns.append(j)
        
        ps[i, j] = p_over_k(i, n, kmax, ps)
        


In [178]:
ps_df = pd.DataFrame(ps)
ps_df

,0,1,2,3,4,5,6,7,8,9,10
0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1,1.0,1.0,2.0,2.0,3.0,3.0,4.0,4.0,5.0,5.0,6.0
2,1.0,1.0,2.0,3.0,4.0,5.0,7.0,8.0,10.0,12.0,14.0
3,1.0,1.0,2.0,3.0,5.0,6.0,9.0,11.0,15.0,18.0,23.0
4,1.0,1.0,2.0,3.0,5.0,7.0,10.0,13.0,18.0,23.0,30.0


In [179]:
ps_df.to_csv("ps.csv", index = False)  

In [ ]:
def fill_ps(max_s, max_n):
    
    ps = np.full(shape = [max_s, max_n + 1], fill_value= np.nan)
    ps[0, range(max_n + 1)] = 1
    ps[range(max_s), 0] = 1
    ps[range(max_s), 1] = 1


    kmax = fill_ks(max_s, max_n)
    
    for i in range(1, ps.shape[0]):    
    for j in range(2, ps.shape[1]):
        
        n = j - 1
        
        ps[i, j] = p_over_k(i, n, kmax, ps)
    
    return(ps)
    